In [ ]:
!pip install tweepy

In [ ]:
import re
import tweepy, json
import pandas as pd
import matplotlib.pyplot as plt
from tweepy import OAuthHandler
from tweepy import Stream
import csv

# keys and tokens from the Twitter Dev Console
consumer_key = '7sK9bZZFhR53gjdx3FlcCguN1'
consumer_secret = 'HbDBpcmkIXFFNjXlCzx3BYFmrVqWUi5H6UIUibxw0IA6TdTWuq'
access_token = '1332372658542022656-epFlmQUJ5WusDSMrjf7FZ2DBfb6qw9'
access_token_secret = 'AFHiYSO5kLn7GTmQ7aDj8cS2tIOhOk9HPDUGYtmILDA0X'
tweet_list = list()

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = api = tweepy.API(auth)

In [ ]:
file = open('tweet.csv', 'a',encoding='utf-8')
csvWriter = csv.writer(file)
for tweet in tweepy.Cursor(api.search, q="foodprice",lang="en", count = 20000).items():
    if tweet.lang == "en" and tweet.user.location =="Malaysia" or "malaysia" or "Singapore" or "singapore":
      csvWriter.writerow([tweet.created_at, tweet.text])
file.close()
# tweets = tweepy.Cursor(api.search, q=["#PriceHike", "#risingprice", '#expensivefood', '#priceIncrease'],lang="en", count = 20000, since="2020-01-01").items()

In [ ]:
data = pd.read_csv("tweet.csv", header = None, encoding='utf-8', names = ['Time', 'Tweets'])

In [ ]:
#removing hashtags related to globalwarming
def rem_hashtags(text):
    processed_text = re.sub(r"#globalwarming", "", text)
    processed_text = " ".join(processed_text.split())
    return processed_text

data['Tweets'] = data['Tweets'].apply(lambda x:rem_hashtags(x))

In [ ]:
#removing tagged users from the tweets
def remove_users(text):
    processed_text = re.sub(r'@\w+ ?',"",text)
    processed_text = " ".join(processed_text.split())
    return processed_text

data['Tweets'] = data['Tweets'].apply(lambda x:remove_users(x))


In [ ]:
#removing hyperlinks mentioned in the tweets
def remove_links(text):
    processed_text = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", text)
    processed_text = " ".join(processed_text.split())
    return processed_text

data['Tweets'] = data['Tweets'].apply(lambda x:remove_links(x))


In [ ]:
#removing punctuations in the tweets
def remove_punct(text):
    punctuations = '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'
    text  = "".join([char for char in text if char not in punctuations])
    text = re.sub('[0-9]+', '', text)
    return text

data['Tweets'] = data['Tweets'].apply(lambda x: remove_punct(x))


In [ ]:
#making all tweets lowercase
def lowercase_word(text):
    text  = "".join([char.lower() for char in text])
    return text
data['Tweets'] = data['Tweets'].apply(lambda x: lowercase_word(x))

In [ ]:
!pip istall textblob

ERROR: unknown command "istall" - maybe you meant "install"


In [ ]:
from textblob import TextBlob

In [ ]:
def get_tweet_sentiment(data):
    if data > 0:
        return 'positive'
    elif data == 0:
        return 'neutral'
    else:
        return 'negative'
tweets = [TextBlob(tweet) for tweet in data['Tweets']]
data['polarity'] = [b.sentiment.polarity for b in tweets]
data['subjectivity'] = [b.sentiment.subjectivity for b in tweets]
data['sentiment'] = data['polarity'].apply(get_tweet_sentiment)
data['sentiment'].value_counts()

positive    69
neutral     22
negative    10
Name: sentiment, dtype: int64

In [ ]:
data.head()

,Time,Tweets,polarity,subjectivity,sentiment
0,2022-06-14 07:28:47,nstnation the best way to curb pricehike of fo...,1.00,0.30,positive
1,2022-06-17 06:43:32,rt he said canteen operators had to increase ...,-0.30,0.40,negative
2,2022-06-17 01:30:31,he said canteen operators had to increase food...,-0.30,0.40,negative
3,2022-06-16 11:52:18,the national bureau of statistics nbs has said...,0.16,0.54,positive
4,2022-06-16 06:23:58,uk foodprice rises could hit over summer soa...,0.00,1.00,neutral


In [ ]:
data.to_csv("result.csv", index=False)